In [1]:
import pandas as pd
from functions import data_preprocess_functions
import glob
import importlib
import numpy as np
from functions import geopack_wrapper

# from pyspedas import time_double
# from pyspedas import time_string
import datetime as datetime

#from functions import pygeo_wrapper

Load IGRF coefficients ...


In [2]:
dir_300_name = 'idl_plots/output_300sec_multi_pa2_en2_subtraction_reduced_removebi_flux0.500.751.00_pap3.02.01.1/'

dir_120_name = 'idl_plots/output_120sec_multi_pa2_en2_subtraction_reduced_removebi_flux0.500.751.00_pap3.02.01.1/data_withoutweight/sc1_sp3/'

### Read and preprocess data averaged over 5 minutes. The data is used for streaming O+ transport path study

In [3]:
importlib.reload(data_preprocess_functions)

dir_name = dir_300_name

data_dir = dir_name + 'data/sc1_sp3/'
tplot_map_dir =  dir_name + 'tplot_map/sc1_sp3/'
output_dir = data_dir


save_to_agg_beam_ext_filename = data_dir + 'aggregated_fulldata_20170101_to_20201231.csv'
save_to_dispersion_full_filename = data_dir + 'dispersion_20170101_to_20201231.csv'

beam_filenames = sorted([i for i in glob.glob(data_dir+"*beam.csv") if i not in glob.glob(dir_name+"storm_o_beam_2016*beam.csv")] )
external_filenames = sorted([i for i in glob.glob(data_dir+"*external.csv") if i not in glob.glob(dir_name+"storm_o_beam_2016*external.csv")])
dispersion_filenames = sorted([i for i in glob.glob(data_dir+"*dispersion.csv") if i not in glob.glob(dir_name+"storm_o_beam_2016*dispersion.csv")])

In [4]:
df_beam = data_preprocess_functions.read_beam_csv(beam_filenames)
df_ext = data_preprocess_functions.read_external_csv(external_filenames)
df_dispersion = data_preprocess_functions.read_dispersion_csv(dispersion_filenames)

df_beam_ext = data_preprocess_functions.preprocess_data(pd.merge(df_beam, df_ext, on = 'time', how='outer' ),remove_large_y=False )
agg_df_beam_ext = data_preprocess_functions.aggregate_energy(data_preprocess_functions.aggregate_angle(df_beam_ext))
agg_df_beam_ext.to_csv(save_to_agg_beam_ext_filename)


ValueError: No objects to concatenate

In [5]:
# read directly from processed file
idl_fulldata_filename = tplot_map_dir + 'tplot_map/fulldata_20170101_to_20201231.csv'
save_to_idl_agg_fulldata_filename = tplot_map_dir + 'idl_aggregated_fulldata_20170101_to_20201231.csv'

df_fulldata0 = pd.read_csv(idl_fulldata_filename)
df_fulldata0 = df_fulldata0.rename(columns=str.lower)
df_fulldata = data_preprocess_functions.preprocess_data(df_fulldata0)

agg_df_fulldata = data_preprocess_functions.aggregate_energy(data_preprocess_functions.aggregate_angle(df_fulldata))
agg_df_fulldata.to_csv(save_to_idl_agg_fulldata_filename)

In [ ]:
# dispersion list
df_dispersion_full = pd.merge(df_dispersion, agg_df_fulldata, on = 'time', how = 'left')
df_dispersion_full = data_preprocess_functions.preprocess_dispersion_list(df_dispersion_full, model = 't89')
df_dispersion_full.to_csv(save_to_dispersion_full_filename)

### Read, combine and preprosess the data for dispersion study (nightside auroral outflowing)

In [ ]:
importlib.reload(data_preprocess_functions)

dir_name = dir_120_name

# save_to_agg_beam_ext_filename = dir_name + 'aggregated_fulldata_20160101_to_20201231.csv'
save_to_dispersion_full_filename = dir_name + 'dispersion_20160101_to_20201231.csv'

dispersion_filenames = sorted(glob.glob(dir_name+"data/*dispersion.csv"))
beam_filenames = [s.replace('dispersion','beam') for s in dispersion_filenames]
external_filenames = [s.replace('dispersion','external') for s in dispersion_filenames] 

df_beam = data_preprocess_functions.read_beam_csv(beam_filenames)
df_ext = data_preprocess_functions.read_external_csv(external_filenames)
df_dispersion = data_preprocess_functions.read_dispersion_csv(dispersion_filenames)

df_beam_ext = data_preprocess_functions.preprocess_data(pd.merge(df_beam, df_ext, on = 'time', how='outer' ),remove_large_y=False )
agg_df_beam_ext = data_preprocess_functions.aggregate_energy(data_preprocess_functions.aggregate_angle(df_beam_ext))
# agg_df_beam_ext.to_csv(save_to_agg_beam_ext_filename)

# read directly from processed file
idl_fulldata_filename = dir_name + 'tplot_map/tplot_map/fulldata_20160101_to_20201231.csv'
save_to_idl_agg_fulldata_filename = dir_name + 'idl_aggregated_fulldata_20160101_to_20201231.csv'

df_fulldata0 = pd.read_csv(idl_fulldata_filename)
df_fulldata0 = df_fulldata0.rename(columns=str.lower)
df_fulldata = data_preprocess_functions.preprocess_data(df_fulldata0)

agg_df_fulldata = data_preprocess_functions.aggregate_energy(data_preprocess_functions.aggregate_angle(df_fulldata))
agg_df_fulldata.to_csv(save_to_idl_agg_fulldata_filename)

df_dispersion_full = pd.merge(df_dispersion, agg_df_fulldata, on = 'time', how = 'left')
df_dispersion_full = df_dispersion_full[df_dispersion_full['kp'] >= 0]
df_dispersion_full = data_preprocess_functions.preprocess_dispersion_list(df_dispersion_full, model = 't89')
df_dispersion_full.to_csv(save_to_dispersion_full_filename)